In [1]:
!git clone https://github.com/workpiles/CUCUMBER-9.git

Cloning into 'CUCUMBER-9'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 13 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
# ファイルを解凍する。
# prototype_1
!tar -zxvf ./CUCUMBER-9/prototype_1/cucumber-9-python.tar.gz

batches.meta
data_batch_5
data_batch_4
data_batch_3
data_batch_2
data_batch_1
test_batch


In [0]:
!mkdir weights

In [4]:
import numpy as np
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, BatchNormalization, LeakyReLU 
from keras.layers import Reshape, Conv2DTranspose, Activation, MaxPooling2D
from keras.layers import Flatten, Dropout, Lambda
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
def load_image_and_label(pickled_files):
    # 各ファイルには495画像が含まれている
    IMAGE_COUNT_PER_FILE = 495
    # 画像の形状 : 32x32x3
    ROW = 32
    COL = 32
    DIM = 3
    whole_images = np.empty((IMAGE_COUNT_PER_FILE*len(pickled_files), ROW, COL, DIM))
    whole_labels = np.empty(IMAGE_COUNT_PER_FILE*len(pickled_files))
    for i, pickled_file in enumerate(pickled_files):
        dict = _unpickle(pickled_file)
        images = dict['data'].reshape(IMAGE_COUNT_PER_FILE, DIM, ROW, COL).transpose(0, 2, 3, 1)
        whole_images[i*IMAGE_COUNT_PER_FILE:(i + 1)*IMAGE_COUNT_PER_FILE, :, :, :] = images
        labels = dict['labels']
        whole_labels[i*IMAGE_COUNT_PER_FILE:(i + 1)*IMAGE_COUNT_PER_FILE] = labels
    return (whole_images, whole_labels)

In [0]:
def _unpickle(pickled_file):
    import pickle

    with open(pickled_file, 'rb') as file:
        dict = pickle.load(file, encoding='latin1')
    return dict

In [0]:
def load_cucumber():
    (X_train, y_train) = load_image_and_label(['data_batch_1',
                                     'data_batch_2',
                                     'data_batch_3',
                                     'data_batch_4',
                                     'data_batch_5',])
    (X_test, y_test) = load_image_and_label(['test_batch'])
    
    return X_train, X_test, y_train, y_test

In [0]:
def normalize(X):
    return (X-127.5)/127.5

In [0]:
### データの準備
X_train, X_test, y_train, y_test = load_cucumber()
X_train = normalize(X_train)
X_test = normalize(X_test)
input_shape = X_train[0].shape
X_test_original = X_test.copy()

y_train = np.eye(9)[y_train.astype('int32').flatten()]
y_test = np.eye(9)[y_test.astype('int32').flatten()]


In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2475, 32, 32, 3)
(2475, 9)
(495, 32, 32, 3)
(495, 9)


In [11]:
model = Sequential()

W0627 01:07:37.641025 139816422119296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [0]:
model.add(Dense(units=256, input_shape=(1024,)))
model.add(Activation('relu'))
model.add(Dense(units=100))
model.add(Activation('relu'))
model.add(Dense(units=10))
model.add(Activation('softmax'))

In [0]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='Adam',
    metrics=['accuracy']
)

In [17]:
early_stopping = EarlyStopping(patience=1, verbose=1)
model.fit(x=X_train, y=y_train, batch_size=256, epochs=10, verbose=1,
          validation_data=(X_test, y_test), callbacks=[early_stopping])

ValueError: ignored

In [0]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.537748224807508
Test accuracy: 0.42828282870427525
